### Read data and visualize
Process
 1. Read data from wikipedia
 2. Create nx.graph
 3. Visuzlize

three options for reading data,
- option 0:directory from python dict
- option 1:from rdb
- option 2:from elasticsearch


### 1. Read data from wikipedia

In [ ]:
import pandas as pd
from wikiscraper import WikiScraper
data = pd.read_csv("contents.csv",index_col=0)
wscs = []

# j = 0
# start = 0
# stop = start+1+len(data.index.values)

for i in data.index.values:
#     j+=1
#     if j<start:
#         continue
        
    wsc = WikiScraper()
    wsc.load(name=i, pageid=data.at[i, 'id'], lang='ja')    
    wscs.append(wsc)
    
    print('-------------------------------------')
    print(wsc)
    print('+++++++++++++++++++++')
#     print wsc._name

#     if j>=stop:
#         break

#to put manually get data
# pd.DataFrame.to_dict(data.T)


### 2.Create nx.graph

three options for reading data,
- option 0:directory from python dict
- option 1:from elasticsearch

In [ ]:
#option 0 directory create nx.graph from python dict
from graph import pre_create_graph, create_graph
input_data = pd.read_csv("contents_test.csv",index_col=0)
points = {}
data = {}
for wsc in wscs:
    points[wsc._name] = float(input_data.at[wsc._name, 'ポイント'])
    data[wsc._name] = wsc._result
    
G = create_graph(pre_create_graph(points=points, data=data))

In [ ]:
#option 1 use elasticsearch
# 1-1 put data into elasticsearch
from elastic_util import ElasticUtilNameId
eu_content = ElasticUtilNameId(index='contents')

i = 0
for wsc in wscs:
    res_dict = wsc._result
    print(i, [wsc._name])
    res_dict['name'] = wsc._name 
    eu_content.put(wsc._name, res_dict)
    i+=1

In [ ]:
#read from elasticsearch
# 1-2 create nx.graph from elasticsearch
import pandas as pd
from graph import pre_create_graph, create_graph
from elastic_util import ElasticUtilNameId
eu_content = ElasticUtilNameId(index='contents')

#1. read name and point from csv
input_data = pd.read_csv("contents_test.csv",index_col=0)

#2. get data from elastic search
names = input_data.index.values
points = {}
data = {}
for name in names:
    points[name] = float(input_data.at[name, 'point'])
    res, code = eu_content.get(name)
    if code==200:
        data[name] = res['_source']
    else:
        print(res, code)
        
G = create_graph(pre_create_graph(points=points, data=data)) 
# for node in G.node():
#     print(node, '\n')

# for debug with swagger
# print(points)


### 3. Visuzlize

1. remove nodes 
2. visualize

In [ ]:
#remove nodes for visualization
from graph import pre_create_graph, create_graph, reduce_node, merge_genre

G2 = reduce_node(G, max_number_of_nodes=500)
G2 = merge_genre(G2)
G = G2

# for node in G.node():
#     print(node, '\n')
# print(G.edges)
# print(G.node['TBS'])
print(list(G.edges))
# G.edges.__str__()

In [ ]:
#visualize with pyvis + networkx 
from pyvis.network import Network
# nxg = nx.complete_graph(10)
# gg = Network(notebook=True)
# gg.from_nx(G)
# gg.show("nx.html")
threshold=1.0

def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)
net.show_buttons(filter_=['physics'])
net.force_atlas_2based(
    gravity=-30, 
    central_gravity=0.01, 
    spring_length=1.0, 
    spring_strength=0.8, 
    damping=1.0, 
    overlap=1.0)
# net.options.physics.__dict__['timestep'] = 0.5
net.options.physics.__dict__['adaptiveTimestep'] = True
net.toggle_hide_edges_on_drag(True)

scaling = dict(min=1, 
                       max=100, 
                       label=dict(enable=True, 
                                          min=10, 
                                          max=100))

node_id = 0
for label in G.nodes:
#     print label, G.node[label]['genre'], G.node[label]['point']
    if G.node[label]['genre'] == 'attribute':
        shape = 'dot'
        value = G.node[label]['point']
        mass=G.node[label]['point']
    else:
        shape = 'box'
        value = threshold
        mass = 10
    net.add_node(node_id, label=label, 
                 group=G.node[label]['genre'], 
                 value=value,
                 mass=mass,
                 scaling = scaling,
                 shape=shape)
    node_id += 1
    
for edge in G.edges:
    edge[0], edge[1]
    net.add_edge( get_net_node(net, 'label', edge[0])['id'], 
                 get_net_node(net, 'label', edge[1])['id'])
    
# net.save_graph("result.html")
net.show("result.html")